# Cournot competition: Oligopoly and monopoly


In our model project we consider a market of goods with Cournot competetion, known from various microeconomic courses. First, we consinder and solve the most Cournot competetion with oligoploly, and later on extend the model to monopoly with taxation

Cournot competition describtions a situation where $N$ number of firms competive on quanteties on the same market for goods. 

In our numerical analysis, we begin by importing relevant packages

In [80]:
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import optimize,arange
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import math
from IPython.display import display, Markdown, clear_output
from traitlets import traitlets


## Cournot - a brief summary

In this section, we describe the setup and solve it theoreticly when 3 firms compete on quanteties on the same market of goods. 
We have take some of the most important assumptions for a Cournot model, the rest can be seen when clicking on 'source':
* $N$ number of firms produce a homogeneous product, in our case we set N=3.
* There is no cooperation between the firms.
* The firms compete in quantities and choose quantities simultaneously.
* The firms are economically rational and act strategically, to maximize profit given their competitors' decisions.

### Cournot Oligopoly

We consider the following market with 3 firms with the total market production and the inverse demand function:
\begin{equation}
p=1-x_1-b(x_2+x_3)
\end{equation}
where 0<b<0.5 both are parameters. 'b' is the rate of substitution. $x_1,x_2,x_3$ is the production of each of the three firms

In [81]:
def demand(x1,x2,x3,b):
    return (1-x1-b*x2-b*x3)

Since all the firms produce the same product and have the same costs, we can define the profit function for firm 1 as:
\begin{equation}
\Pi_1=px_1-cx_1
\end{equation}
where $px_1$ is the total revenue of the production and $cx_1$ is the total cost of the production, note c is the marginal cost and we have that $0\le c\le 0,5.$


In [82]:
#Here we define the cost function
def cost(x,c):
    if x == 0:
     cost = 0
    else:
     cost = c*x
    return cost

#Here we define the profit function and state that if profit is less than 0
#then it is better to produce nothing. 

def profit(x1,x2,x3,c1,b):
    profit = demand(x1,x2,x3,b)*x1 -cost(x1,c1)
    if profit < 0:
        profit == 0
    else:
        profit = demand(x1,x2,x3,b)*x1 -cost(x1,c1)
    
    return profit

We now find the first order condition, (FOC), and afterwards calculate the best response function, (BR), of firm 1, given the production of firm 2 and 3.  
FOC:
\begin{equation}
\frac{\partial \Pi_1}{\partial x_1}=0 
\longleftrightarrow 1-bx_1-bx_2-bx_3-c=0 
\end{equation}
We can now find the BR function of firm 1, given the production of firm 2 and frim 2:
\begin{equation}
1-bx_2-bx_3-c=2x_1 \leftrightarrow x_1 =\big(\frac{1-c-bx_2-bx_3}{2}\big) 
\end{equation}
The BR function can therefore be defined as:
\begin{equation}
BR_1(x_2,x_3)=\big(\frac{1-c-bx_2-bx_3}{2}\big)
\end{equation}

Note: In our code, we have defined the BR function as "reaction".

In [83]:
def reaction(x2,x3,c1,b):
    x1 = optimize.brute(lambda x: -profit(x,x2,x3,c1,b), ((-10,10,),), Ns=100) #optimize.brute allways minimize. Therefore-
    # we want to use the function so we minimize the negative profit which means that we maximize the profit. 
    return x1[0]      

def vector_reaction(x,param): # We create a function where the vector of x* (which is to total-
    #output of each firm) - f(x*) is equal to zero. This is because we want to find the equilibrium where each firm produce the output in demand.  
    return array(x)-array([reaction(x[0],x[2],param[1],param[0]), reaction(x[0],x[1],param[1],param[0]),
                          reaction(x[1],x[2],param[2],param[0])])

#  param is a list of parameters, param = (b1,c1,c2,c3).

We can now find the Nash Equilibrium on the market, and since we have symmetry we have:
\begin{equation}
x_1=x_2=x_3=x^*
\end{equation}
Given symmetry we find the produced quantity of goods as:
\begin{equation}
x^*= \big(\frac{1-c-bx^*-bx^*}{2}\big)
\end{equation}
This is reduced to:
\begin{equation}
x^*=\frac{1-c}{4}
\end{equation}

In the next two instances of our code, we will assign both the marginal cost imposed on given firms, but also the degree of substitution between the goods these firms produce. 
The substituion value of 1 gives us the goods are perfect substitutes while 0.0 gives us the goods are perfect compliments.

In [84]:
sub = widgets.FloatSlider(    #Here we define a interaktive widgets
    value = 1.00,   
    min = 0.0,
    max = 1.00,
    step = 0.01,
    description = 'Subsitution:',
    disabled = False,
    continuous_update=True,
    orientation = 'horizontal',
    readout=True,
    readout_format='.2f'
    
)
cost1 = widgets.BoundedFloatText(
    value = 0.00,
    min = 0.00,
    max = 0.50,
    step = 0.01,
    description = 'Cost firm 1',
    disabled = False,
    continuous_update = True,
    readout = True,
    readout_format = '.2f'
)

cost2 = widgets.BoundedFloatText(
    value = 0.00,
    min = 0.00,
    max = 0.50,
    step = 0.01,
    description = 'Cost firm 2',
    disabled = False,
    continuous_update = True,
    readout = True,
    readout_format = '.2f'
)

cost3 = widgets.BoundedFloatText(
    value = 0.00,
    min = 0.00,
    max = 0.50,
    step = 0.01,
    description = 'Cost firm 3',
    disabled = False,
    continuous_update = True,
    readout = True,
    readout_format = '.2f'
)

The cost for firms is also limited to between 0 and 0.5 to avoid any potential error in our solution.

In [85]:
#Defining values for each firm
widgets.HBox([widgets.VBox([sub, cost1, cost2, cost3])])

We can now insert this solution into our inverse demand function to determine the price:
\begin{equation}
p=1-x_1-b(x_2+x_3) 
\leftrightarrow
\end{equation}

\begin{equation}
p=1-b(\frac{1-c}{4b}+\frac{1-c}{4b}+\frac{1-c}{4b})
\leftrightarrow
\end{equation}

\begin{equation}
p=1-(\frac{1-c}{4}+\frac{1-c}{4}+\frac{1-c}{4})
\leftrightarrow
\end{equation}

\begin{equation}
p=1-(\frac{3}{4}-\frac{3c}{4})
\leftrightarrow 
\end{equation}

\begin{equation}
p=1-\frac{3}{4}+\frac{3c}{4} \leftrightarrow 
\end{equation}

\begin{equation}
p=\frac{4}{4}-\frac{3}{4}+\frac{3c}{4}  \leftrightarrow  
\end{equation}

\begin{equation}
p=\frac{1+3c}{4} 
\end{equation}


An impornant note about the next section. The values for substitutions and costs are updated continiously. As such, when these values are changed the next lines of code has to be refreshed. Once refreshed all you have to do is press "Calculate" and you will get a value.

In [86]:
#The solution part 

param = [sub.value, cost1.value, cost2.value, cost3.value]
x0 = [0.0, 0.0, 0.0]   # this is our initial "guess" of the production x*

ans = optimize.fsolve(vector_reaction, x0, args = (param))

button = widgets.Button(description='Calculate Price')
out = widgets.Output() 


caption = widgets.Label(value='If you set b to 0, the model will solve it as 3 monopoly firms, independtent of each, as they operate on different markets')
caption1 = widgets.Label(value='Remember to refresh the code (The solution part) above if you change the values of b or c1,c2,c3')

def on_button_clicked(b):
    with out:
        refresh_output()
        print(sub)


def on_button_clicked(b):
    with out:
        clear_output()
        print(ans)

# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([caption,caption1,button,out])

Now that we have solved the Cournot oligopoly both theoretical and with help of Python, we will in the next part assume the three firms merge into one big firm.

### Cournot Monopoly

We now assume all three firms merge into one big firm. First, we define inverse demand function of the newly established firm:

\begin{equation}
p=1-x
\end{equation}
x is the total production of the firm.

In [74]:
def demand(x1):
    return (1-x1)

def cost(x,c):
    if x == 0:
     cost = 0
    else:
     cost = c*x
    return cost

The revenue for the firm will be defined as:
\begin{equation}
R=p*x=(1-x)*x
\end{equation}

In [75]:
def revenue(x1,c1):
    return demand(x1)*x1 -cost(x1,c1)

Since we work with monopoly, we know that the marginal revenue, MR, always will be equal to the marginal cost,MC. First we define the marginal revenue
\begin{equation}
MR=\frac{\partial R}{\partial X}=0 
\leftrightarrow
\end{equation}
\begin{equation}
MR=1-2X
\end{equation}


In [76]:
def reaction(c1):
    x1 = optimize.brute(lambda x: -revenue(x,c1), ((0,1,),), Ns=10000) # brute minimizes the function;
                                                                 # Its is the same as above
    return x1[0]                             

In [77]:
def vector_reaction(x,param): # param = (c1)
    return array(x)-array([reaction(param[0])])  

We now define the marginal cost:
\begin{equation}
MC=c
\end{equation}

We now set MR=MC

\begin{equation}
MR=MC
\leftrightarrow
\end{equation}

\begin{equation}
1-2X=c \leftrightarrow
\end{equation}

\begin{equation}
1-c=2bX \leftrightarrow
\end{equation}

\begin{equation}
\frac{1-c}{2}=X
\end{equation}

\begin{equation}
X^{Monopoly}=\frac{1-c}{2}
\end{equation}

As we are working with monopoly now, we do not chose whenever a product is a perfect substitute or perfect compliment, as we are only considering one good. For now we just define 'b' as a parameter, the only important factor is how much it cost the monopoly firm to produce the given good.

In [78]:
costM = widgets.BoundedFloatText(
    value = 0.00,
    min = 0.00,
    max = 0.50,
    step = 0.01,
    description = 'Monopoly cost',
    disabled = False,
    continuous_update = True,
    readout = True,
    readout_format = '.2f'
)
costM

BoundedFloatText(value=0.0, continuous_update=True, description='Monopoly cost', max=0.5, step=0.01)

We can now identify the monopoly price by inserting the monopoly quantity in the inverse demand function:
\begin{equation}
p^{Monopoly}=1-X^{Monopoly} 
\leftrightarrow
\end{equation}

\begin{equation}
p^{monopoly}=1-\frac{1+c}{2}
\leftrightarrow
\end{equation}

\begin{equation}
p^{monopoly}= \frac{1+c}{2}
\end{equation}

Note: Same rules apply to this button as to the last one. That said, there is a minor bug, where when you press the previous button this button will show its result. Once this new button is pressed the values update to correct monopoly values

In [79]:
param = [costM.value]
x0 = [0.0]

ansM = optimize.fsolve(vector_reaction, x0, args = (param))


button = widgets.Button(description='Calculate Price')
out = widgets.Output() 

def on_button_clicked(b):
    with out:
        refresh_output() 
        print(sub)


def on_button_clicked(b):
    with out:
        clear_output()
        print(ansM)

# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

Now that we done both the theoretical and nummerical analysis for Cournot competition, we come up with the final remarks.
When we compare the price for monopoly vs. oligopoly, we see the monopoly price is the highest (for the case when c=0 in both scenarios).